In [ ]:
import pandas as pd
import requests
from datetime import timezone
import datetime

In [11]:
from datetime import timezone
import datetime

# Getting the current date and time
dt = datetime.datetime.now()
print(dt)


2025-05-06 13:35:23.319662


In [12]:
query = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-05-05T12:00:00&endtime=2025-05-06T12:00:00'
data = requests.get(query)

In [15]:
len(data.json()['features'])

208